# Local Models vs Model Chains

This notebook compares the scores of the local models and the two model chains (where we propagate the true values of the predictions).

In [20]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score
from chaining import Chain
import os

In [21]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [22]:
def missingness_and_categorical_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Stratify categorical variables
    for col in df.select_dtypes(include=['category']):
        counts = df[col].value_counts(normalize=True)
        for category in counts.index:
            idx = df[col] == category
            cv[idx] = cv[idx].fillna(np.random.choice(np.where(idx)[0], size=int(counts[category] * N_FOLDS), replace=False))

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [23]:
# Insert path to data file here
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# File name
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

data = pd.read_csv(path)

# Bin the number of relapses into 0, 1, 2, 3 and 4+ 
def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

# Display all columns
pd.set_option('display.max_columns', None)
#data

In [24]:
# Choice of target variables, and listed already in the chain order 
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'EDSS-after_2y', 'NRELAP', 'CESEV']

In [25]:
# Extract targets
targets = data[variables]

# Choice of input variables
columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE', 
                    'NHPT-before', 'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'T25FW-before', 'SLEC_before','SES_before',
                    'BDI-before', 'EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before',
                	'P_R36-SF12-before', 'R36-SF12-before_Ind', 'T-before','P-before','N-before']

features = data[columns_to_keep]
#features

In [26]:
# Use one-hot encoding for categorical and binary input variables
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
#features.head()

In [27]:
targets.dtypes

KFSS_M-2y           float64
KFSS_P-2y           float64
EDSS-2y             float64
T25FW-2y            float64
NHPT-2y             float64
P_R36-SF12-after    float64
M_R36-SF12-after    float64
SES_after           float64
SLEC_after          float64
KFSS_M-after_2y     float64
KFSS_P-after_2y     float64
EDSS-after_2y       float64
NRELAP               object
CESEV                object
dtype: object

In [28]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [29]:
# Generate CV folds
cv=missingness_and_categorical_stratified_cv(targets, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

targets_cv['CV Fold'].value_counts()

CV Fold
4.0    510
3.0    502
0.0    500
1.0    495
2.0    458
Name: count, dtype: int64

In [30]:
# Group by CV fold value
grouped = targets_cv.groupby('CV Fold')

# Calculate the percentage of missing values for each variable in each CV fold
missing_percentages = grouped.apply(lambda x: x.isna().mean() * 100)
missing_percentages

C:\Users\anaso\AppData\Local\Temp\ipykernel_40924\1893073231.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_percentages = grouped.apply(lambda x: x.isna().mean() * 100)


,KFSS_M-2y,KFSS_P-2y,EDSS-2y,T25FW-2y,NHPT-2y,P_R36-SF12-after,M_R36-SF12-after,SES_after,SLEC_after,KFSS_M-after_2y,KFSS_P-after_2y,EDSS-after_2y,NRELAP,CESEV,CV Fold
CV Fold,,,,,,,,,,,,,,,
0.0,44.200000,44.200000,33.000000,30.800000,33.200000,36.600000,36.600000,66.000000,66.000000,77.000000,77.000000,72.800000,0.0,64.000000,0.0
1.0,48.888889,48.888889,36.363636,33.737374,36.767677,39.191919,39.191919,68.888889,68.888889,80.606061,80.606061,76.161616,0.0,62.222222,0.0
2.0,44.541485,44.541485,34.061135,31.877729,34.061135,37.991266,37.991266,64.410480,64.410480,78.820961,78.820961,74.890830,0.0,58.515284,0.0
3.0,48.406375,48.406375,34.262948,31.872510,34.262948,37.250996,37.250996,71.314741,71.314741,75.896414,75.896414,70.517928,0.0,62.749004,0.0
4.0,48.039216,48.039216,34.901961,33.333333,35.098039,38.823529,38.823529,70.000000,70.000000,80.196078,80.196078,74.509804,0.0,61.568627,0.0


In [31]:
# Calculate the relative percentage count of each level of NRELAP and CESEV within each CV fold
cat_percentage_counts = grouped.apply(lambda x: x[['NRELAP', 'CESEV']].apply(lambda y: y.value_counts(normalize=True) * 100))
cat_percentage_counts

C:\Users\anaso\AppData\Local\Temp\ipykernel_40924\672282790.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cat_percentage_counts = grouped.apply(lambda x: x[['NRELAP', 'CESEV']].apply(lambda y: y.value_counts(normalize=True) * 100))


NRELAP      CESEV
CV Fold                               
0.0     0.0       67.400000        NaN
        1.0       17.600000        NaN
        2.0        7.400000        NaN
        3.0        3.800000        NaN
        4+         3.800000        NaN
        MILD            NaN  20.000000
        MODERATE        NaN  60.000000
        SEVERE          NaN  20.000000
1.0     0.0       65.454545        NaN
        1.0       18.585859        NaN
        2.0        9.090909        NaN
        3.0        3.636364        NaN
        4+         3.232323        NaN
        MILD            NaN  25.668449
        MODERATE        NaN  60.427807
        SEVERE          NaN  13.903743
2.0     0.0       62.663755        NaN
        1.0       20.960699        NaN
        2.0       10.917031        NaN
        3.0        2.401747        NaN
        4+         3.056769        NaN
        MILD            NaN  22.631579
        MODERATE        NaN  60.526316
        SEVERE          NaN  16.842105
3.0     0.0       65.139442        NaN
        1.0       20.517928        NaN
        2.0        8.565737        NaN
        3.0        2.788845        NaN
        4+         2.988048        NaN
        MILD            NaN  29.411765
        MODERATE        NaN  53.475936
        SEVERE          NaN  17.112299
4.0     0.0       66.078431        NaN
        1.0       20.784314        NaN
        2.0        7.058824        NaN
        3.0        3.921569        NaN
        4+         2.156863        NaN
        MILD            NaN  28.571429
        MODERATE        NaN  50.000000
        SEVERE          NaN  21.428571

---

# Local Models

In [32]:
y_pred_list = []
y_test_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate=False, #RUN LOCAL MODELS
    )
    
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [33]:
# Remove rows in y_test and y_pred where the variable in question is missing in y_test (since without it, it is not possible to calculate the score)
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

1st index: fold, 2nd index: outcome

In [34]:
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables:
    variable_scores = []
    
    # Scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] 
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))


print("Scores for each outcome (local):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.6f} (± {std_score:.6f})")

Scores for each outcome (local):
KFSS_M-2y: 0.806110 (± 0.024257)
KFSS_P-2y: 0.747655 (± 0.035666)
EDSS-2y: 0.881740 (± 0.012522)
T25FW-2y: 0.723295 (± 0.082169)
NHPT-2y: 0.487602 (± 0.100730)
P_R36-SF12-after: 0.692193 (± 0.018844)
M_R36-SF12-after: 0.566317 (± 0.025900)
SES_after: 0.673104 (± 0.061261)
SLEC_after: 0.661404 (± 0.048492)
KFSS_M-after_2y: 0.652697 (± 0.051669)
KFSS_P-after_2y: 0.508382 (± 0.075025)
EDSS-after_2y: 0.756476 (± 0.023314)
NRELAP: 0.637156 (± 0.010758)
CESEV: 0.498147 (± 0.035706)


---

# Model Chain

### Propagate predicted values

In [35]:
y_pred_list = []
y_test_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred", 
    )
    
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [36]:
# Remove rows in y_test and y_pred where the variable in question is missing in y_test (since without it, it is not possible to calculate the score)
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)

In [37]:
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] 
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))


print("Scores for each outcome (chain - predicted values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.6f} (± {std_score:.6f})")

Scores for each outcome (chain - predicted values):
KFSS_M-2y: 0.806110 (± 0.024257)
KFSS_P-2y: 0.748826 (± 0.038452)
EDSS-2y: 0.878399 (± 0.009057)
T25FW-2y: 0.739289 (± 0.068823)
NHPT-2y: 0.543487 (± 0.106313)
P_R36-SF12-after: 0.693806 (± 0.021025)
M_R36-SF12-after: 0.539594 (± 0.043098)
SES_after: 0.675966 (± 0.048457)
SLEC_after: 0.657846 (± 0.050603)
KFSS_M-after_2y: 0.644624 (± 0.051360)
KFSS_P-after_2y: 0.496987 (± 0.079715)
EDSS-after_2y: 0.747596 (± 0.027036)
NRELAP: 0.637414 (± 0.015700)
CESEV: 0.490859 (± 0.038634)


### Propagate true values

In [38]:
y_pred_list = []
y_test_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="true",
    )
    
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_list.append(pd.DataFrame(y_pred, columns=yi_test.columns, index=yi_test.index))
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [39]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)

In [40]:
# Initialize a list to store scores
scores = []
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Scores for the variable across all folds
    for fold_index in range(len(y_test_cv)):
        y_test = y_test_cv[fold_index][variables.index(variable_name)] 
        y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
        
        # Check if the target variable is numerical or categorical
        if y_test.dtype.kind in 'bifc':
            score = r2_score(y_test, y_pred)
        else:
            score = accuracy_score(y_test, y_pred)
                  
        variable_scores.append(score)
    
    # Average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    # Standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))


print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.6f} (± {std_score:.6f})")

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.806110 (± 0.024257)
KFSS_P-2y: 0.748199 (± 0.035516)
EDSS-2y: 0.882074 (± 0.009358)
T25FW-2y: 0.743260 (± 0.081836)
NHPT-2y: 0.547253 (± 0.112405)
P_R36-SF12-after: 0.701295 (± 0.017075)
M_R36-SF12-after: 0.548614 (± 0.041483)
SES_after: 0.677078 (± 0.061710)
SLEC_after: 0.657925 (± 0.043438)
KFSS_M-after_2y: 0.657765 (± 0.057367)
KFSS_P-after_2y: 0.507064 (± 0.051130)
EDSS-after_2y: 0.758362 (± 0.027480)
NRELAP: 0.634280 (± 0.020465)
CESEV: 0.563588 (± 0.042535)
